In [1]:
import numpy as np
import pandas as pd
import time
import parallelPermutationTest as ppt
from scipy.stats import ttest_ind, mannwhitneyu

In [2]:
%load_ext cython
%load_ext rpy2.ipython

In [3]:
%%R

require(ggplot2)
install.packages("devtools")
library("devtools")
install_github("bdsegal/fastPerm")

library(fastPerm)

R[write to console]: Loading required package: ggplot2

R[write to console]: Installing package into ‘/home/ekvall/R/x86_64-pc-linux-gnu-library/4.0’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/devtools_2.3.1.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 373604 bytes (364 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: 

In [ ]:
url = "https://static-content.springer.com/esm/art%3A10.1038%2Fnature18003/MediaObjects/41586_2016_BFnature18003_MOESM111_ESM.zip"
!mkdir "data/"
!wget -P "data/" "https://static-content.springer.com/esm/art%3A10.1038%2Fnature18003/MediaObjects/41586_2016_BFnature18003_MOESM111_ESM.zip"
!unzip "./data/41586_2016_BFnature18003_MOESM111_ESM.zip" -d "./data/"

In [22]:
path = "./data/nature18003-s2/"
T1 = pd.read_excel(path + "CPTAC_BC_SupplementaryTable01.xlsx")
T3 = pd.read_excel(path + "CPTAC_BC_SupplementaryTable03.xlsx")

## Extract important column headings

In [23]:
paiteint_id = T1.columns[0]
stats_col = T1.columns[5: 5 + 3]

### Patient column  

In [24]:
paiteint_id

'Sample ID'

### Cancer phenotype coulmns

In [25]:
stats_col

Index(['ER Status', 'PR Status', 'HER2 Status'], dtype='object')

### Check viable phenotype status

In [26]:
all_lab = list()
for r in T1[stats_col].iterrows():
    all_lab += list(r[1].values)

In [27]:
list(set(all_lab))

['Negative', 'Indeterminate', 'Equivocal', 'Positive']

### Let's drop patient with "Equivocal" and "Indeterminate" status

In [28]:
drop_label = ['Indeterminate', 'Equivocal']

In [29]:
T1.head()

,Sample ID,TCGA ID,Biospecimen Barcode Side,UIUD,PAM50,ER Status,PR Status,HER2 Status,QC Status,Proteome Cluster (see Fig. 3b),...,iTRAQReporterIon,CommonControl,GATA3 Mutation,PIK3CA Mutation,TP53 Mutation,PIK3CA missense mutation in helical domain all tumors,PIK3CA missense mutation in kinase domain all tumors,TP53 Nonsense/Frameshift all tumors,TP53 Missense mutation all tumors,TP53 missense mutations in DNA binding domain all tumors
0,A2-A0CM,TCGA-A2-A0CM,TCGA-A2-A0CM-01A-11-A21V-30,330F7598-824C-4CD6-9303-A27FE74A6695,Basal,Negative,Negative,Negative,pass,1.0,...,116,YES,NaN,NaN,Frame_Shift_Del|Somatic|p.E204fs,0.0,0.0,1.0,NaN,NaN
1,A2-A0D2,TCGA-A2-A0D2,TCGA-A2-A0D2-01A-12-A21W-30,308CCD79-C164-4397-92BC-A1CD243C8E7D,Basal,Negative,Negative,Negative,pass,1.0,...,114,NO,NaN,NaN,Frame_Shift_Del|Somatic|p.P318fs,0.0,0.0,1.0,NaN,NaN
2,A2-A0EQ,TCGA-A2-A0EQ,TCGA-A2-A0EQ-01A-41-A21W-30,04217F65-2907-478B-B0C5-EB65370198DA,Her2,Negative,Negative,Positive,pass,1.0,...,116,NO,NaN,Missense_Mutation|Somatic|p.H1047R,In_Frame_Del|Somatic|p.IY162in_frame_delN,NaN,1.0,NaN,NaN,NaN
3,A2-A0EV,TCGA-A2-A0EV,TCGA-A2-A0EV-01A-41-A21V-30,38F98E5F-7FA4-4C89-8D81-516CF865BEEB,LumA,Positive,Positive,Negative,pass,3.0,...,114,NO,NaN,In_Frame_Ins|Somatic|p.E469in_frame_insDK,NaN,NaN,NaN,0.0,0.0,0.0
4,A2-A0EX,TCGA-A2-A0EX,TCGA-A2-A0EX-01A-41-A21V-30,78271500-147B-40C5-B789-0CF0C7CDBCE2,LumA,Positive,Positive,Negative,pass,3.0,...,116,NO,NaN,Missense_Mutation|Somatic|p.E545K,NaN,1.0,NaN,0.0,0.0,0.0


In [30]:
mask1 = [True if (len(set(r[1].values) & set(drop_label)) == 0) else False for r in T1[stats_col].iterrows()]
T1 = T1[mask1]
T1.reset_index(drop=True, inplace=True)

### Let's divide the dataframe into two dataframes with triple negative (TN) and non-triple negative (NTN)

In [31]:
mask2 = [all([True if v=='Negative' else False for v in r[1].values] ) for r in T1[stats_col].iterrows()]

T1_TNP = T1[np.array(mask2)]
T1_TNP.reset_index(drop=True, inplace=True)

T1_not_TNP = T1[~np.array(mask2)]
T1_not_TNP.reset_index(drop=True, inplace=True)



### Obtain patient ids for NT and NTN

In [32]:
TNPpateintIds = T1_TNP[paiteint_id].values
NotTNPpateintIds = T1_not_TNP[paiteint_id].values

### Remove genes (rows) containing NaN for both TN and NTN i.e., make sure all patients have same the gene when comparing.

In [33]:
def getpatientId(ids):
    patitentList = list()
    for i in ids:
        p_id = T3[T3.columns[[i in c for c in T3.columns]]].columns
        patitentList += list(p_id)
    return patitentList

In [34]:
only_patientDf = T3[getpatientId(NotTNPpateintIds) + getpatientId(TNPpateintIds)]

In [35]:
only_patientDf.dropna(inplace=True)

/home/ekvall/anaconda3/envs/terran3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
NotTNP_df =  only_patientDf[getpatientId(NotTNPpateintIds)]
TNP_df =  only_patientDf[getpatientId(TNPpateintIds)]

In [37]:
NotTNP_df.to_csv("experiment_data/experiment6/notTNPdf", index=False)
TNP_df.to_csv("experiment_data/experiment6/TNPdf", index=False)

In [38]:
NotTNP_df = pd.read_csv("experiment_data/experiment6/notTNPdf")
TNP_df = pd.read_csv("experiment_data/experiment6/TNPdf")

In [39]:
TNP_Arr = TNP_df.values
NotTNP_Arr = NotTNP_df.values

In [40]:
x = pd.DataFrame(TNP_Arr, columns=list(range(TNP_Arr.shape[1]))).T
y = pd.DataFrame(NotTNP_Arr, columns=list(range(NotTNP_Arr.shape[1]))).T
n_samples = x.shape[1]
outputDf = pd.DataFrame(n_samples * [0], columns=["output"]).T

In [41]:
%R -i x
%R -i y
%R -i outputDf
%R -i n_samples

In [42]:
%%R
runFastPerm <- function(x,y,output, n_samples) {
    e <- list(mode="vector",length=n_samples)
        for (i in 1:n_samples) {
      
            mStopDiffMean(x[[i]], y[[i]])
            valX = fastPerm(x[[i]], y[[i]], testStat = diffMean)
            
            print(valX)
            valX <-unlist(valX)

      
            output[[i]] <- as.numeric(valX[1])
      
            }
    

    return(output)


}


In [43]:
start = time.time()
%R out <- runFastPerm(x,y, outputDf, n_samples)
end = time.time()
print(end - start)

    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.0769
p-value = 0.794
mStop = 20, deviance = 184, AIC = 365
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.0734
p-value = 0.798
mStop = 20, deviance = 197, AIC = 379
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.0706
p-value = 0.808
mStop = 20, deviance = 171, AIC = 353
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.076
p-value = 0.789
mStop = 20, deviance = 196, AIC = 378
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.0742
p-value = 0.809
mStop = 20, deviance = 194, AIC = 376
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.073
p-value = 0.806
mStop = 20, de

    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.74
p-value = 0.000463
mStop = 17, deviance = 337, AIC = 432
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.731
p-value = 0.000515
mStop = 15, deviance = 291, AIC = 381
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.724
p-value = 0.00071
mStop = 18, deviance = 326, AIC = 426
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.307
p-value = 0.284
mStop = 20, deviance = 496, AIC = 663
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.0465
p-value = 0.925
mStop = 20, deviance = 48.8, AIC = 234
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.147
p-value = 0.63
mStop = 20

    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 1.65
p-value = 0.000238
mStop = 15, deviance = 359, AIC = 446
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 1.71
p-value = 4.51e-05
mStop = 13, deviance = 248, AIC = 323
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 1.76
p-value = 2.13e-05
mStop = 12, deviance = 261, AIC = 332
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.762
p-value = 0.0353
mStop = 20, deviance = 358, AIC = 504
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.335
p-value = 0.326
mStop = 20, deviance = 505, AIC = 673
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.147
p-value = 0.793
mStop = 20,

    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.483
p-value = 0.197
mStop = 20, deviance = 541, AIC = 703
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.503
p-value = 0.134
mStop = 20, deviance = 448, AIC = 607
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.267
p-value = 0.035
mStop = 20, deviance = 425, AIC = 570
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 1.5
p-value = 9.03e-05
mStop = 15, deviance = 244, AIC = 327
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.979
p-value = 0.0155
mStop = 20, deviance = 393, AIC = 530
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.178
p-value = 0.271
mStop = 20, devi

    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.016
p-value = 0.968
mStop = 20, deviance = 7.2, AIC = 194
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.811
p-value = 0.000442
mStop = 17, deviance = 361, AIC = 455
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.499
p-value = 0.0026
mStop = 20, deviance = 297, AIC = 415
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.501
p-value = 0.000755
mStop = 19, deviance = 309, AIC = 413
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.0414
p-value = 0.914
mStop = 20, deviance = 74.4, AIC = 259
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.488
p-value = 0.233
mStop = 2

...

    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.125
p-value = 0.803
mStop = 20, deviance = 143, AIC = 325
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.547
p-value = 0.448
mStop = 20, deviance = 473, AIC = 645
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.804
p-value = 0.00404
mStop = 20, deviance = 373, AIC = 495
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.127
p-value = 0.423
mStop = 20, deviance = 489, AIC = 660
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.224
p-value = 0.69
mStop = 20, deviance = 284, AIC = 462
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.0269
p-value = 0.962
mStop = 20, devi

    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.311
p-value = 0.204
mStop = 20, deviance = 569, AIC = 731
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.389
p-value = 0.497
mStop = 20, deviance = 459, AIC = 632
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.435
p-value = 0.0423
mStop = 20, deviance = 390, AIC = 537
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.13
p-value = 0.745
mStop = 20, deviance = 211, AIC = 392
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.000598
p-value = 0.999
mStop = 20, deviance = 0.038, AIC = 188
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.512
p-value = 0.0735
mStop = 20, 

    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.0237
p-value = 0.973
mStop = 20, deviance = 9.74, AIC = 197
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 1.2
p-value = 0.00357
mStop = 20, deviance = 360, AIC = 480
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.393
p-value = 0.258
mStop = 20, deviance = 543, AIC = 708
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.0747
p-value = 0.822
mStop = 20, deviance = 246, AIC = 428
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.23
p-value = 0.474
mStop = 20, deviance = 444, AIC = 617
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.00398
p-value = 0.996
mStop = 20, de

    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.143
p-value = 0.893
mStop = 20, deviance = 103, AIC = 287
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.185
p-value = 0.581
mStop = 20, deviance = 345, AIC = 521
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.712
p-value = 0.389
mStop = 20, deviance = 481, AIC = 651
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.0709
p-value = 0.928
mStop = 20, deviance = 42.1, AIC = 227
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.485
p-value = 0.697
mStop = 20, deviance = 243, AIC = 422
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.543
p-value = 0.064
mStop = 20, devi

    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.437
p-value = 0.0723
mStop = 20, deviance = 389, AIC = 541
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.975
p-value = 0.00199
mStop = 20, deviance = 304, AIC = 421
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.462
p-value = 0.286
mStop = 20, deviance = 553, AIC = 719
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 1.05
p-value = 0.331
mStop = 20, deviance = 498, AIC = 666
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.0773
p-value = 0.897
mStop = 20, deviance = 81.8, AIC = 266
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 1.68
p-value = 1.43e-06
mStop = 11, 

In [44]:
round(2477.251361131668 / 60,2)

41.29

### There are 80 patients without triple negative (NTN) and 26 patients with triple negtive (TN), and with a total of 8051 quantified genes.

# Experiment time

## Let's start with a rather small binsize=6.

In [18]:
%time p_val_bin_8 = ppt.GreenFloatCuda(TNP_Arr, NotTNP_Arr, 8)

CPU times: user 1.17 s, sys: 107 ms, total: 1.28 s
Wall time: 1.28 s


In [20]:
%time p_val_bin_16 = ppt.GreenFloatCuda(TNP_Arr, NotTNP_Arr, 16)

CPU times: user 1.34 s, sys: 180 ms, total: 1.52 s
Wall time: 1.52 s


In [8]:
%time p_val_bin_32 = ppt.GreenFloatCuda(TNP_Arr, NotTNP_Arr, 32)

CPU times: user 1.9 s, sys: 320 ms, total: 2.22 s
Wall time: 2.21 s


In [9]:
%time p_val_bin_64 = ppt.GreenFloatCuda(TNP_Arr, NotTNP_Arr, 64)

CPU times: user 3 s, sys: 412 ms, total: 3.41 s
Wall time: 3.41 s


In [10]:
%time p_val_bin_128 = ppt.GreenFloatCuda(TNP_Arr, NotTNP_Arr, 128, batch_size = TNP_Arr.shape[0] / 2)

CPU times: user 5.29 s, sys: 652 ms, total: 5.94 s
Wall time: 5.92 s


In [11]:
%time p_val_bin_256 = ppt.GreenFloatCuda(TNP_Arr, NotTNP_Arr, 256, batch_size = TNP_Arr.shape[0] / 3)

CPU times: user 9.7 s, sys: 1.13 s, total: 10.8 s
Wall time: 10.8 s


In [12]:
%time p_val_bin_512 = ppt.GreenFloatCuda(TNP_Arr, NotTNP_Arr, 512, batch_size = TNP_Arr.shape[0] / 6)

CPU times: user 18.4 s, sys: 2.28 s, total: 20.7 s
Wall time: 20.7 s


In [13]:
%time p_val_bin_1280 = ppt.GreenFloatCuda(TNP_Arr, NotTNP_Arr, 1028, batch_size = TNP_Arr.shape[0] / 12)

CPU times: user 35.7 s, sys: 4.11 s, total: 39.8 s
Wall time: 39.8 s


In [14]:
%time p_val_bin_2056 = ppt.GreenFloatCuda(TNP_Arr, NotTNP_Arr, 2056, batch_size = TNP_Arr.shape[0] / 24)

CPU times: user 1min 10s, sys: 8.43 s, total: 1min 19s
Wall time: 1min 19s


### Parallelized exact test takes ~3s

In [15]:
def MWU(A, B, one_side=False):
    p_mw = list()
    for a,b in zip(A, B):
        if one_side:
            p_mw.append(mannwhitneyu(a,b, alternative="less")[1])
        else:
            p_mw.append(mannwhitneyu(a,b, alternative="two-sided")[1])
    return p_mw

In [21]:
%time p_mwu = MWU(TNP_Arr, NotTNP_Arr, one_side=False)

CPU times: user 1.52 s, sys: 3.89 ms, total: 1.53 s
Wall time: 1.52 s


### Mann-Whitney exact test takes ~1s

In [17]:
%time p_ttest = ttest_ind(TNP_Arr.T, NotTNP_Arr.T)[1]

CPU times: user 1.39 ms, sys: 4.17 ms, total: 5.56 ms
Wall time: 5.56 ms
